In [98]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

airlines = pd.read_csv("airlines.csv", names=['Avia company Name','Airport owner','Airport','AircraftType'])
airports = pd.read_csv("airports.csv", names=['Airport','Register certificate','Airport owner','AirportType'])
aircraft = pd.read_csv("aircraft.csv", sep=';')
cargo = pd.read_csv("cargo transportation.csv", sep=';', decimal=",")
passenger = pd.read_csv("passenger transportation.csv", sep=';')

#create predicate
aircraft_type = aircraft['Вид воздушного судна'] == 'самолет'

aircraft1 = aircraft[aircraft_type]
aircraft1["Тип (наименование) воздушного судна"].unique()

array(['Птенец-2', 'С-2 Елочка', 'Небесный', ..., 'Ил-96-300ПУ(М1)',
       'Ил-96-400Т/IL96-400T', 'Ил-96-400 ВПУ'], dtype=object)

In [52]:
date = pd.to_datetime(aircraft['дата действующего свидетельства о регистрации']).min()
dateTime = aircraft['дата действующего свидетельства о регистрации'] == date.strftime("%d.%m.%Y")
aircraft3 = aircraft[dateTime]
aircraft3['Тип (наименование) воздушного судна']

6635    Ан-2
Name: Тип (наименование) воздушного судна, dtype: object

In [109]:
def normolize_name(li_air):
    li_0 = li_air[0].split()
    new_list = []
    if len(li_air)>1 and len(li_0)>1:
        li_0 = li_air[0].split()
        if (li_0[1].isalpha()) and (len(li_0[1])>1):
            li_0 = ' '.join(li_0[:2])
            new_list.append(li_0)
        else:
            new_list.append(li_0[0])
    else:
        if li_air[0].count('(')!=0:
            li_air = re.sub(r'\s+', r'', li_air[0])
        else:
            li_air = re.sub(r'\s+', r'', li_0[0])
        new_list.append(li_air)
    return ''.join(new_list)

passenger = passenger.replace('***',0)
passenger_df = passenger.drop(columns = 'Наименование аэропорта РФ')
passenger_df = passenger_df.iloc[:, 1:].apply(lambda x: x.str.replace(' ','')).fillna(0)
passenger_df = passenger_df.apply(pd.to_numeric)
passenger_df['Год периода данных'] = passenger['Год периода данных']
passenger_df['Наименование аэропорта РФ'] =passenger['Наименование аэропорта РФ']

passenger_df['Наименование аэропорта РФ'] = (
    passenger_df['Наименование аэропорта РФ'].str.split().apply(lambda x:normolize_name(x)))
cargo = cargo.replace('***',0)
cargo_df = cargo.drop(columns = 'Наименование аэропорта РФ')
cargo_df = cargo_df.apply(lambda x: x.str.replace(',','.')).apply(lambda x: x.str.replace(' ','')).fillna(0)
cargo_df = cargo_df.apply(pd.to_numeric)
cargo_df['Наименование аэропорта РФ'] =cargo['Наименование аэропорта РФ']
cargo_df['Наименование аэропорта РФ'] = (
    cargo_df['Наименование аэропорта РФ'].str.split().apply(lambda x:normolize_name(x)))
airports[airports['Airport'].isna()]
airports = airports.dropna()
airports['Airport'] = airports['Airport'].str.split(',').apply(lambda x:normolize_name(x))
airports['AirportType']=airports['AirportType'].str.replace(r'\s+','')
passenger_df['Пассажиропоток']=passenger_df.iloc[:, :-2].sum(1)
cargo_df['Сумма грузоперевозок'] = cargo_df.iloc[:, 1:-1].sum(1)

pd.merge(
    airports[['Airport','Airport owner']],
    pd.merge(cargo_df[cargo_df['Год периода данных']==2018][['Наименование аэропорта РФ','Сумма грузоперевозок']],
    passenger_df[passenger_df['Год периода данных']==2018][['Наименование аэропорта РФ','Пассажиропоток']],
         how='left'),
    how='right'
    ,left_on='Airport',right_on='Наименование аэропорта РФ')#.drop(columns='Аэропорт')


,Airport,Airport owner,Наименование аэропорта РФ,Сумма грузоперевозок,Пассажиропоток
0,Архангельск(Талаги),ОАО «Аэропорт Архангельск»,Архангельск(Талаги),1385.60,732453
1,Нарьян-Мар,ОАО «Нарьян-Марский объединенный авиаотряд»,Нарьян-Мар,0.00,0
2,Котлас,ОАО «Аэросервис»,Котлас,0.00,0
3,Лешуконское,ОАО «Аэропорт Лешуконское»,Лешуконское,0.00,0
4,Амдерма,ФКП «Аэропорт Амдерма»,Амдерма,0.00,0
5,Мезень,ОАО «2-ой Архангельский ОАО»,Мезень,0.00,0
6,Нижняя,ОАО «Нарьян-Марский ОАО,Нижняя,0.00,0
7,Соловки,ОАО «2-ой Архангельский ОАО»,Соловки,0.00,0
8,Несь,ОАО «Нарьян-Марский ОАО»,Несь,0.00,0
9,Усть-Кара,ОАО «Нарьян-Марский ОАО»,Усть-Кара,0.00,0


In [125]:
pas_df = passenger_df.iloc[:,-2:].groupby('Наименование аэропорта РФ').agg('sum').reset_index()
carg_df = cargo_df.iloc[:,-2:].groupby('Наименование аэропорта РФ').agg('sum').reset_index()

median_cargo = carg_df[carg_df['Сумма грузоперевозок'] != 0]['Сумма грузоперевозок'].median()
median_passen = pas_df[pas_df['Пассажиропоток'] != 0]['Пассажиропоток'].median()

tabl = pd.merge(carg_df[['Наименование аэропорта РФ','Сумма грузоперевозок']],
    pas_df[['Наименование аэропорта РФ','Пассажиропоток']])

(tabl[(tabl['Сумма грузоперевозок'] > median_cargo) & (tabl['Пассажиропоток'] < median_passen) ]
[['Наименование аэропорта РФ']].sort_values('Наименование аэропорта РФ'))


,Наименование аэропорта РФ
0,Абакан
4,Анадырь
93,Когалым
96,Комсомольск-на-Амуре
134,Москва(Шереметьево)-Карго
154,Новокузнецк
181,Певек
195,Полярный
276,Хатанга
303,Ярославль(Туношна)


In [158]:
airlines = pd.read_csv("airlines.csv", names=['Avia company Name','Airport owner','Airport','AircraftType'])

airlines = airlines.dropna()
airlines['AircraftType'] = (airlines['AircraftType']
                                                   .str.replace(r'\s+\([0-9]+\)','')
                                                   .str.replace(r'\s+','')
                                                   .str.split(',')
                                                  )

airlines[~airlines['AircraftType'].apply(lambda x: 
                                         any(i in aircraft['Тип (наименование) воздушного судна'].values for i in x)
                                        )][['Avia company Name']]

,Avia company Name
481,Авиа Менеджмент Груп
486,Аврора
533,Ай Флай
499,АТРАН
505,Аэролимузин
1,Аэрофлот ОАО
451,ВИМ-АВИА
491,Джет Эйр Групп
89,ДОНАВИА
36,Икар ООО


In [162]:
passenger_pddf = passenger_d.groupby(['Наименование аэропорта РФ']).agg(sum).reset_index().iloc[:,:-2]
list_months = pd.DataFrame(passenger_pddf.iloc[:,1:].sum(0)).rename_axis(['Месяц']).reset_index()
list_months = list_months.rename(columns={0:'Пассажироперевозки в месяц'})
airports_list=[]
for i in range(pas.iloc[:,1:].max(axis=0).shape[0]):
    name = passenger_pddf[pas.iloc[:, i+1] == passenger_pddf.iloc[:,1:].max(axis=0)[i]]['Наименование аэропорта РФ']
    airports_list.append(name[name.index[0]])
list_months['Наименование аэропорта РФ']=airports_list
list_months

,Месяц,Пассажироперевозки в месяц,Наименование аэропорта РФ
0,Январь,92581956,Москва(Шереметьево)
1,Февраль,84279197,Москва(Шереметьево)
2,Март,97548315,Москва(Шереметьево)
3,Апрель,103999255,Москва(Шереметьево)
4,Май,120740351,Москва(Домодедово)
5,Июнь,146081971,Москва(Домодедово)
6,Июль,167043305,Москва(Домодедово)
7,Август,171378013,Москва(Домодедово)
8,Сентябрь,148372441,Москва(Домодедово)
9,Октябрь,108301174,Москва(Шереметьево)


In [179]:
cargo_agg = cargo_df.iloc[:,1:-1].groupby('Наименование аэропорта РФ').agg('mean').reset_index()
cargo_agg['средний грузопоток в месяц в течние года'] = cargo_agg.mean(axis=1)
(pd.merge(
    airports[['Airport','AirportType']],cargo_agg,
    left_on='Airport',right_on='Наименование аэропорта РФ', how='inner').drop(columns='Airport')
    .drop(columns = "Наименование аэропорта РФ")
    .groupby('AirportType').agg('mean')).reset_index()


,AirportType,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь,средний грузопоток в месяц в течние года
0,B,50.583333,61.416667,67.833333,69.416667,60.250000,66.000000,62.000000,68.833333,70.250000,69.083333,67.583333,81.750000,749.916667,118.839744
1,А,548.914526,692.590317,819.938719,852.911439,822.535907,845.650128,891.090245,985.097992,1026.150441,1027.974045,944.916899,987.243456,10154.435048,1584.573012
2,А-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,А-2,174.153333,224.041667,245.757500,268.923333,241.490833,262.619167,265.960000,275.333333,302.177500,289.959167,280.059167,351.584167,3167.150833,488.400769
4,Б,218.015880,292.895370,313.322731,336.854907,341.806435,329.488519,328.265880,357.508704,357.800185,362.703056,364.454028,402.909815,3812.200417,601.401994
5,Б(горный),90.526667,126.913333,119.017083,155.510000,158.971667,107.702083,114.945000,119.994583,167.117500,203.101667,88.732083,121.046667,1467.398750,233.921314
6,Б-1,30.192083,40.633333,42.966667,45.412917,46.045417,50.641667,49.451667,46.185417,46.831667,40.677500,36.212917,52.936667,420.935000,73.009455
7,В,46.971955,61.133301,69.001715,79.688285,81.406667,79.392388,77.327420,78.272115,83.396891,82.005433,82.905689,92.202356,874.664760,137.566844
8,Г,1.238836,1.744722,1.982341,2.396243,2.876296,3.131918,2.182262,2.883029,2.115463,2.286138,2.044577,2.925569,26.698585,4.192768
9,Г(горный),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
